In [19]:
import pandas as pd

orders = pd.read_csv('orders.csv')
orders["Timestamp"] = pd.to_datetime(orders["Timestamp"])
orders.columns = ['timestamp', 'group id', 'side', 'price', 'size']

orders["group id"] = orders["group id"].astype('int')
orders["price"] = orders["price"].astype('float')
orders["size"] = orders["size"].astype('int')

orders["order id"] = orders.index + 1
orders.head()

,timestamp,group id,side,price,size,order id
0,2024-09-30 20:26:01,5,buy,8.0,50,1
1,2024-09-30 20:26:11,14,sell,11.0,20,2
2,2024-09-30 20:26:14,10,sell,11.0,100,3
3,2024-09-30 20:26:15,12,sell,10.0,50,4
4,2024-09-30 20:26:16,7,buy,5.0,50,5


## Main structure of CLOB
- deal with each order and try to match a successful trade
- search the ideal prices, from good to bad, then from ealry to late
- place in a proper position if not matched / partly matched
- update the CLOB
- record in the trade book (trade_df)

In [20]:
# rename to bid & ask in the final display
clob = {"buy": {}, "sell": {}}
trade_df = pd.DataFrame(columns=["timestamp", "buy", "sell", "price", "size"]) 

def restart():
    global clob, trade_df
    clob = {"buy": {}, "sell": {}}
    trade_df = pd.DataFrame(columns=["timestamp", "buy id", "sell id", "price", "size"]) 

In [21]:
def upload_clob_info(order, outstanding=""):
    if not outstanding:
        outstanding = order["size"]
    if type(outstanding) != int:
        outstanding = int(outstanding)
    price0 = float(order["price"])
    order_dic = {"time":order["timestamp"],
                 "order id": int(order["order id"]), 
                 "size": int(order["size"]),
                 }
    if price0 not in clob[order["side"]]:
        clob[order["side"]][price0] = [order_dic]
    else:
        clob[order["side"]][price0].append(order_dic)

def upload_trade_info(trades):
    global trade_df
    # dict -> df
    # "pre-existing","latecomer","trade size" -> "timestamp", "buy", "sell", "price", "size"
    for j in trades:
        buyer_seller = {orders.iloc[j["latecomer"]-1, 2]: orders.iloc[j["latecomer"]-1, 1],
                        orders.iloc[j["pre-existing"]-1, 2]: orders.iloc[j["pre-existing"]-1, 1]}
        trade_detail =  [orders.iloc[j["latecomer"]-1, 0],
                        buyer_seller["buy"],
                        buyer_seller["sell"],
                        orders.iloc[j["pre-existing"]-1, 3],
                        j["trade size"]]
        new_df = pd.DataFrame([trade_detail], columns=trade_df.columns)
        if trade_df.empty:
            trade_df = new_df
        else:
            trade_df = pd.concat([trade_df, new_df], ignore_index=True)

def get_order_feature_at_the_price(target_feature, order0=pd.Series(dtype='float64'), price="", side="", opp=False):
    if not ((not order0.empty) or (price and side)):
        print("Error: input either order or side&price")
        return None
    elif not order0.empty:
        side, price = order0["side"], float(order0["price"])
    else:
        price = float(price)
    if opp:
        side = "buy" if side == "sell" else "sell"
    orders_at_the_price = clob[side][price]
    try:
        return [i[target_feature] for i in orders_at_the_price]
    except KeyError:
        print("Error: no such feature")
        return None

In [22]:
def match_trade(order):
    # return outstanding, matched_trades
    matched_trades = []

    side, price, outstanding = order["side"], float(order["price"]), int(order["size"])
    opp_side = "buy" if side == "sell" else "sell"
    opp_side_price = clob[opp_side].keys()
    if side == "buy":
        good_prices = [p for p in opp_side_price if p <= price and sum(get_order_feature_at_the_price("size",price=p,side="buy",opp=True)) > 0]
        if not good_prices:
            return outstanding, []
        good_prices.sort() # buy side look for the lowest price

    elif side == "sell":
        good_prices = [p for p in opp_side_price if p >= price and sum(get_order_feature_at_the_price("size",price=p,side="sell",opp=True)) > 0]
        if not good_prices:
            return outstanding, []
        good_prices.sort(reverse=True) # sell side look for the highest price
    
    for good_price in good_prices:
        for i, opp_order in enumerate(clob[opp_side][good_price]):
            opp_size = opp_order["size"]
            if outstanding == 0:
                return 0, matched_trades
            if outstanding < 0:
                raise ValueError("outstanding should not be negative")
            if opp_size == 0:
                continue
            if opp_size >= outstanding:
                matched_trades.append({"pre-existing":int(opp_order["order id"]),
                                        "latecomer": int(order["order id"]),
                                        "trade size": outstanding})
                clob[opp_side][good_price][i]["size"] -= outstanding # assign the value in the dictionary
                return 0, matched_trades
            else:
                outstanding -= opp_size
                clob[opp_side][good_price][i]["size"] = 0 # assign the value in the dictionary
                matched_trades.append({"pre-existing":int(opp_order["order id"]),
                                        "latecomer": int(order["order id"]),
                                        "trade size": opp_size})
        return outstanding, matched_trades
    

## Order Book State

In [23]:
# clob


## Trade Log

In [24]:
# trade_df

## Spread and Market Depth

In [25]:
# Print the bid-ask spread in the loop
def print_spread():
    highest_bid = max(clob["buy"].keys()) if clob["buy"] else None # 还需要筛选sum size>0
    lowest_ask = min(clob["sell"].keys()) if clob["sell"] else None # 还需要筛选sum size>0
    if highest_bid and lowest_ask:
        return lowest_ask - highest_bid

In [26]:
# Loop through all trades
restart()
for i in range(0, len(orders)):
#for i in range(0, 30): # for testing
    order = orders.iloc[i]
    outstanding, matched_trades = match_trade(order)
    if outstanding > 0:
        # Place the rest in the Order Book
        upload_clob_info(order, outstanding)
    if matched_trades:
        upload_trade_info(matched_trades)
    
    
    # Print the updated records
    bid_ask_spread = print_spread()
        



In [27]:
clob

{'buy': {8.0: [{'time': Timestamp('2024-09-30 20:26:01'),
    'order id': 1,
    'size': 50},
   {'time': Timestamp('2024-09-30 20:26:29'), 'order id': 9, 'size': 50},
   {'time': Timestamp('2024-09-30 20:27:39'), 'order id': 20, 'size': 50},
   {'time': Timestamp('2024-09-30 20:32:41'), 'order id': 27, 'size': 50},
   {'time': Timestamp('2024-09-30 20:42:15'), 'order id': 93, 'size': 10}],
  5.0: [{'time': Timestamp('2024-09-30 20:26:16'), 'order id': 5, 'size': 50},
   {'time': Timestamp('2024-09-30 20:41:58'), 'order id': 87, 'size': 80}],
  7.0: [{'time': Timestamp('2024-09-30 20:26:27'),
    'order id': 8,
    'size': 142}],
  10.0: [{'time': Timestamp('2024-09-30 20:26:47'), 'order id': 15, 'size': 0},
   {'time': Timestamp('2024-09-30 20:26:48'), 'order id': 16, 'size': 0},
   {'time': Timestamp('2024-09-30 20:33:46'), 'order id': 37, 'size': 0},
   {'time': Timestamp('2024-09-30 20:41:58'), 'order id': 85, 'size': 220}],
  9.0: [{'time': Timestamp('2024-09-30 20:31:04'),
    'o

In [28]:
trade_df

,timestamp,buy id,sell id,price,size
0,2024-09-30 20:26:34,11,12,10.00,40
1,2024-09-30 20:26:39,9,12,10.00,10
2,2024-09-30 20:26:39,9,4,10.00,50
3,2024-09-30 20:26:47,3,4,10.00,10
4,2024-09-30 20:27:20,1,14,11.00,10
5,2024-09-30 20:31:47,3,18,10.00,50
6,2024-09-30 20:31:47,13,18,10.00,50
7,2024-09-30 20:32:59,3,16,9.50,50
8,2024-09-30 20:33:46,15,16,9.50,50
9,2024-09-30 20:33:48,9,20,10.00,40
